In [1]:
! pip install rouge_score
! pip install -U accelerate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d322e066efff099100536923aa0d979808cc0f55928fcb95e832a2c67a13ab21
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.1 MB/s eta 0:00:00


In [2]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00


In [3]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [4]:
import glob
import wandb
import matplotlib.pyplot as plt

from io import StringIO
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch
from datasets import Dataset
from sklearn.metrics import f1_score
import numpy as np
import os

In [5]:
import accelerate

In [6]:
tqdm.pandas()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
os.environ["WANDB_PROJECT"] = "ClinicalBioBERT" # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint" # log all model checkpoints

In [9]:
def read_paper(path):
  f = open(path, 'r', encoding="utf-8")
  text = str(f.read())
  f.close()
  return text

In [10]:
def create_list(folder_path):
  temp_papers = []
  for filename in tqdm(glob.glob(folder_path + "/*.txt")):
      temp_papers.append(read_paper(filename))
  for filename in tqdm(glob.glob(folder_path + "/*.xml")):
      temp_papers.append(read_paper(filename))
  return temp_papers

In [11]:
train_path = '/content/drive/MyDrive/train'
val_path = '/content/drive/MyDrive/validation'
test_path = '/content/drive/MyDrive/test'

In [12]:
list_train = create_list(train_path)
list_val = create_list(val_path)
list_test = create_list(test_path)

100%|██████████| 86/86 [00:00<00:00, 868.00it/s]


In [13]:
list_train[1]

"Admission Date:  [**2133-3-28**]       Discharge Date:  [**2133-4-2**]\n\nDate of Birth:   [**2072-12-20**]       Sex:  M\n\nService:  [**Company 191**]\n\nHISTORY OF PRESENT ILLNESS:  This is a 60-year-old man with\nCAD, PVD who had difficulty speaking after left carotid\nendarterectomy in [**2133-2-16**].  The patient has had bilateral\ncarotid stenosis.  On [**2-21**] while he was sitting in a chair he\ndeveloped sudden onset of right arm and leg numbness,\nfollowed by right arm and leg weakness.  He also had\ndifficulty speaking.  He was admitted to [**Hospital3 **]\nand underwent a left carotid endarterectomy on [**2-26**] and\nafterwards began having severe left sided headache behind his\nleft eye that lasted for hours and was constant.\nNevertheless, he visited [**Hospital3 **] for continued\nheadaches and nausea and vomiting.  During one of those\nvisits he had a contortion of his right face and bilateral\narm jerking and was started on Dilantin with presumptive\ndiagnosis of 

In [14]:
list_train[-1]

'<?xml version="1.0" encoding="UTF-8" ?>\n<PatientMatching>\n<TEXT><![CDATA[\n\nRecord date: 2097-06-09\n\n\n \n\n\nRenal Consult Note\nQuinton Delong,M.D.\n\n\n\n\nCC/ID: Asked to assit in the mangement of Lori Stevens with hypertension, nausea and vomiting.\n\n\n\n\nHISTORY OF PRESENT ILLNESS:\n\n\n\nShe is a 62 year old lady with ESRD who has been admitted in the past with nausea, vomiting and severe hypertension. She was treated with BP medications. She has had a CT scan and is seen in dialysis. She is stable on dialysis currently.\n\n\n\n\n\n\nPAST MEDICAL HISTORY:\n\n1.       Diabetes Mellitus x many years.\n2.       History of hypertension x many years.\n3.       History of GI bleeding.\n4.       ESRD secondary to DM on HD x years. On HD Mon/Wed/Fri.\n5.       CAD s/p NSTEMI.\n6.       Meningitis in the past. ( Viral)\n      \n\n\nMEDICATIONS\n\nUnavailable\n\n\nSOCIAL HISTORY:\n\n\nLives in Nursing Home.\n\n\n\n\nPHYSICAL EXAMINATION:\n\n\n\n\nBP: 159/70  P:  72   R:  16    T=9

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [16]:
model = "emilyalsentzer/Bio_ClinicalBERT"

tokenizer = AutoTokenizer.from_pretrained(model)

model_compactbiobert = AutoModelForMaskedLM.from_pretrained(model).to(device)

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [17]:
encoded_data_train = tokenizer.batch_encode_plus(
    list_train,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding='longest',
    max_length=512,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    list_val,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding='longest',
    max_length=512,
    return_tensors='pt'
)

In [18]:
print(tokenizer)

BertTokenizerFast(name_or_path='emilyalsentzer/Bio_ClinicalBERT', vocab_size=28996, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [19]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15, return_tensors="pt")

In [20]:
savePath = "/content/drive/MyDrive/models/"

In [21]:
trainingArguments = TrainingArguments(
    savePath + "checkpoints",
    logging_steps=1,
    overwrite_output_dir=True,
    num_train_epochs=100,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    do_train=True,
    do_eval=True,
    warmup_steps=5000,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=128,
    metric_for_best_model='eval_loss',
    weight_decay=1e-4,
    fp16=True,
    report_to="wandb",
    gradient_checkpointing=True,
    optim="adamw_torch",
    save_total_limit=2,
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    eval_steps = 1,
    save_strategy = "steps",
    load_best_model_at_end=True
)

In [22]:
dataset_train = Dataset.from_dict(encoded_data_train)
dataset_val = Dataset.from_dict(encoded_data_val)

In [23]:
trainer = Trainer(model=model_compactbiobert, args=trainingArguments,
                  tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset=dataset_train,eval_dataset=dataset_val)

In [24]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
1,0.311400,1.893341
2,0.316000,1.872042
3,0.313200,1.896220
4,0.317300,1.915774
5,0.312800,1.946533
6,0.313900,1.914346
7,0.030300,1.923850


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model(savePath + "ClinicalBioBERT/")